# Sys check

In [1]:
!pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!pip install timm
!pip install matplotlib
!pip install opencv-python
!pip install scikit-learn
!pip install Pillow

In [3]:
import os
root = os.getcwd()
root

'/content'

In [4]:
!nvidia-smi

Sat Apr 12 15:54:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cu124
True


In [13]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [14]:
!unzip /content/drive/MyDrive/dataset.zip #Replace with the path of your data


Archive:  /content/drive/MyDrive/dataset.zip
   creating: dev/
   creating: dev/spoof/
  inflating: dev/spoof/757_1.jpg     
  inflating: dev/spoof/431_2.jpg     
  inflating: dev/spoof/796_1.jpg     
  inflating: dev/spoof/1652_2.jpg    
  inflating: dev/spoof/1484_4.jpg    
  inflating: dev/spoof/1112_4.jpg    
  inflating: dev/spoof/656_1.jpg     
  inflating: dev/spoof/1364_2.jpg    
  inflating: dev/spoof/482_1.jpg     
  inflating: dev/spoof/270_1.jpg     
  inflating: dev/spoof/1165_1.jpg    
  inflating: dev/spoof/1145_3.jpg    
  inflating: dev/spoof/297_4.jpg     
  inflating: dev/spoof/172_3.jpg     
  inflating: dev/spoof/1250_4.jpg    
  inflating: dev/spoof/1787_4.jpg    
  inflating: dev/spoof/418_3.jpg     
  inflating: dev/spoof/581_2.jpg     
  inflating: dev/spoof/278_1.jpg     
  inflating: dev/spoof/278_2.jpg     
  inflating: dev/spoof/108_4.jpg     
  inflating: dev/spoof/1597_4.jpg    
  inflating: dev/spoof/458_1.jpg     
  inflating: dev/spoof/628_3.jpg     
 

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report

import os
import matplotlib.pyplot as plt

In [6]:
def training(model, optimizer, criterion, train_loader, val_loader, num_epochs=10, device='cuda'):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"[Epoch {epoch+1}/{num_epochs}] Loss: {running_loss / len(train_loader):.4f}")

        # Evaluate on validation set
        print("Validation performance:")
        evaluate(model, val_loader)

In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean/std
                         std=[0.229, 0.224, 0.225])
])

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 2
batch_size = 8
num_epochs = 10
learning_rate = 1e-4

In [11]:
# Hàm đánh giá
def evaluate(model, dataloader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    report = classification_report(y_true, y_pred, target_names=["normal", "spoof"])
    print(report)

# Multi-image

In [12]:
from torch.utils.data import Dataset
from PIL import Image
import glob
import os
import torch

class MultiImageLivenessDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Custom dataset for multi-image liveness detection.

        Args:
            root_dir (str): Root directory containing subdirectories for each class ('normal' and 'spoof').
            transform (callable, optional): Transformations to apply to the images.
        """
        self.samples = []  # List to store image paths and labels
        self.transform = transform  # Transformations to apply to images

        # Iterate over the two classes: 'normal' and 'spoof'
        for label_dir in ["normal", "spoof"]:
            full_path = os.path.join(root_dir, label_dir)  # Full path to the class directory
            persons = {}  # Dictionary to group images by person ID

            # Group images by person ID
            for img_path in glob.glob(os.path.join(full_path, "*.jpg")):
                filename = os.path.basename(img_path)  # Extract the filename
                person_id = filename.split("_")[0]  # Extract the person ID from the filename
                if person_id not in persons:
                    persons[person_id] = []  # Initialize a list for the person ID
                persons[person_id].append(img_path)  # Add the image path to the person's list

            # Process each person's images
            for person_id, images in persons.items():
                images = sorted(images)  # Sort images for consistency
                if len(images) >= 4:
                    selected = images[:4]  # Select the first 4 images if there are enough
                else:
                    # If fewer than 4 images, duplicate the first image to make up the difference
                    selected = (images + [images[0]] * 4)[:4]
                # Append the selected images and label (0 for 'normal', 1 for 'spoof') to the samples list
                self.samples.append((selected, 0 if label_dir == "normal" else 1))

    def __len__(self):
        """
        Returns the total number of samples in the dataset.
        """
        return len(self.samples)

    def __getitem__(self, idx):
        """
        Retrieves a sample from the dataset.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            tuple: A tuple containing a tensor of stacked images and the corresponding label.
        """
        img_paths, label = self.samples[idx]  # Get the image paths and label for the given index
        imgs = []  # List to store the processed images

        # Load and process each image
        for path in img_paths:
            image = Image.open(path).convert("RGB")  # Open the image and convert it to RGB
            if self.transform:
                image = self.transform(image)  # Apply transformations if specified
            imgs.append(image)  # Add the processed image to the list

        # Stack the images into a single tensor and return it along with the label
        return torch.stack(imgs), torch.tensor(label)


In [16]:
train_set = MultiImageLivenessDataset("/content/train", transform=transform)
val_set = MultiImageLivenessDataset("/content/dev", transform=transform)
train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
val_loader = DataLoader(val_set, batch_size=8, shuffle=False)

In [17]:
# Hàm đánh giá
def evaluate(model, dataloader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    report = classification_report(y_true, y_pred, target_names=["normal", "spoof"])
    print(report)

## Resnet


In [18]:
import torch.nn as nn
import torchvision.models as models

class MultiImageResNetLSTM(nn.Module):
    def __init__(self, base_model, lstm_hidden=512, num_classes=2):
        """
        A model combining ResNet for feature extraction and LSTM for temporal modeling.

        Args:
            base_model: Pretrained ResNet model.
            lstm_hidden: Number of hidden units in the LSTM layer.
            num_classes: Number of output classes for classification.
        """
        super(MultiImageResNetLSTM, self).__init__()
        # Use ResNet as a feature extractor, removing the fully connected (FC) layer
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-1])  # ResNet50 without FC

        # LSTM layer for temporal modeling of features
        self.lstm = nn.LSTM(input_size=2048, hidden_size=lstm_hidden, batch_first=True)

        # Fully connected classifier
        self.classifier = nn.Sequential(
            nn.Linear(lstm_hidden, 256),  # Linear layer to reduce dimensions
            nn.ReLU(),                   # Activation function
            nn.Dropout(0.4),             # Dropout for regularization
            nn.Linear(256, num_classes)  # Final layer for classification
        )

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x: Input tensor of shape [B, 4, C, H, W], where:
               B = Batch size, 4 = Number of images per sample,
               C = Channels, H = Height, W = Width.

        Returns:
            Output tensor of shape [B, num_classes].
        """
        # Reshape input to process all images in the batch
        B, N, C, H, W = x.shape
        x = x.view(B * N, C, H, W)  # [B * 4, C, H, W]

        # Extract features using ResNet
        features = self.feature_extractor(x)  # [B * 4, 2048, 1, 1]
        features = features.view(B, N, -1)    # Reshape to [B, 4, 2048]

        # Pass features through LSTM
        _, (hn, _) = self.lstm(features)      # hn: [1, B, lstm_hidden]
        hn = hn.squeeze(0)                    # Remove the first dimension: [B, lstm_hidden]

        # Classify using the fully connected layers
        out = self.classifier(hn)             # [B, num_classes]
        return out


In [19]:
resnet = models.resnet50(pretrained=True)
model = MultiImageResNetLSTM(resnet).to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 184MB/s]


In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
training(model, optimizer, criterion, train_loader, val_loader, num_epochs=5, device='cuda')

[Epoch 1/5] Loss: 0.4774
Validation performance:
              precision    recall  f1-score   support

      normal       0.89      0.84      0.86       201
       spoof       0.85      0.90      0.87       204

    accuracy                           0.87       405
   macro avg       0.87      0.87      0.87       405
weighted avg       0.87      0.87      0.87       405

[Epoch 2/5] Loss: 0.3265
Validation performance:
              precision    recall  f1-score   support

      normal       0.84      0.97      0.90       201
       spoof       0.97      0.82      0.89       204

    accuracy                           0.90       405
   macro avg       0.90      0.90      0.90       405
weighted avg       0.91      0.90      0.90       405

[Epoch 3/5] Loss: 0.2664
Validation performance:
              precision    recall  f1-score   support

      normal       0.86      0.94      0.90       201
       spoof       0.93      0.85      0.89       204

    accuracy                       

Evaluate

In [22]:
evaluate(model, val_loader)

              precision    recall  f1-score   support

      normal       0.95      0.92      0.93       201
       spoof       0.92      0.95      0.94       204

    accuracy                           0.94       405
   macro avg       0.94      0.94      0.94       405
weighted avg       0.94      0.94      0.94       405



In [23]:
from sklearn.metrics import roc_auc_score

def calculate_roc_auc(model, dataloader, device):
    model.eval()
    y_true, y_scores = [], []

    with torch.no_grad():
        for image_group, labels in dataloader:
            image_group = image_group.to(device)
            labels = labels.to(device)

            outputs = model(image_group)
            probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Get probabilities for the positive class

            y_true.extend(labels.cpu().numpy())
            y_scores.extend(probabilities.cpu().numpy())

    roc_auc = roc_auc_score(y_true, y_scores)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    return roc_auc

# Calculate ROC-AUC for the validation set
roc_auc = calculate_roc_auc(model, val_loader, device)

ROC-AUC Score: 0.9633


In [ ]:
# # Save the model's state dictionary
# torch.save(model.state_dict(), "resnet50_liveness_multi.pth")

In [ ]:
# # Ensure the model is initialized correctly
# resnet = models.resnet50(pretrained=True)
# model = MultiImageResNetLSTM(resnet).to(device)

# # Load the state dictionary
# model.load_state_dict(torch.load("resnet50_liveness_multi.pth"))

d:\Anaconda\envs\Takehometest\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\envs\Takehometest\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

Predict

In [24]:
from PIL import Image
import torch

def predict_spoof_probability(input_path, model, transform, device):
    """
    Predict if an image or group of images is normal or spoof using the multi-image model.

    Args:
        input_path: Can be either:
                   - Single image path (string)
                   - List of image paths for the same person
                   - Directory containing images of the same person
        model: Trained MultiImageResNetLSTM model
        transform: Preprocessing transforms
        device: Device to run inference on ('cuda' or 'cpu')

    Returns:
        A probability in range [0,1] where:
        - 0 indicates normal (real face)
        - 1 indicates spoof (fake face)
    """
    model.eval()

    # Determine input type
    if isinstance(input_path, str):
        if os.path.isfile(input_path):
            # Single image path
            img_paths = [input_path]
        elif os.path.isdir(input_path):
            # Directory of images
            img_paths = sorted(glob.glob(os.path.join(input_path, "*.jpg")))
            if not img_paths:
                raise ValueError(f"No images found in directory: {input_path}")
        else:
            raise ValueError(f"Input path does not exist: {input_path}")
    elif isinstance(input_path, list):
        # List of image paths
        img_paths = input_path
    else:
        raise ValueError("Input path must be a string or list of strings")

    # Process images
    processed_images = []
    for path in img_paths:
        image = Image.open(path).convert("RGB")
        if transform:
            image = transform(image)
        processed_images.append(image)

    # Handle case where we have fewer than 4 images
    if len(processed_images) < 4:
        # Duplicate first image to get 4 total images
        while len(processed_images) < 4:
            processed_images.append(processed_images[0])
    elif len(processed_images) > 4:
        # If more than 4 images, use only the first 4
        processed_images = processed_images[:4]

    # Stack images and prepare for model
    image_batch = torch.stack(processed_images).unsqueeze(0).to(device)  # [1, 4, C, H, W]

    # Make prediction
    with torch.no_grad():
        outputs = model(image_batch)
        probabilities = torch.softmax(outputs, dim=1)

    # Return spoof probability (index 1)
    return probabilities[0, 1].item()



# Example paths (replace with actual paths)
single_image = "/content/dev/normal/2_4.jpg" #Change with your image path
score1 = predict_spoof_probability(single_image, model, transform, device)
print(f"Single image spoof probability: {score1:.4f}")
print(f"Single image classification: {'Spoof (fake)' if score1 > 0.5 else 'Normal (real)'}")


multiple_images = [
    "/content/dev/spoof/63_1.jpg", #Change with your imagepaths List
    "/content/dev/spoof/63_2.jpg",
    "/content/dev/spoof/63_3.jpg",
    "/content/dev/spoof/63_4.jpg"
]
score2 = predict_spoof_probability(multiple_images, model, transform, device)
print(f"Multiple images spoof probability: {score2:.4f}")
print(f"Multiple images classification: {'Spoof (fake)' if score2 > 0.5 else 'Normal (real)'}")


# person_dir = "D:\\IT\\GITHUB\\TakeHomeTest\\dataset\\dev\\spoof\\person_a"
# # Example directory (replace with actual directory)
# score3 = predict_spoof_probability(person_dir, model, transform, device)
# print(f"Directory spoof probability: {score2:.4f}")
# print(f"Directory classification: {'Spoof (fake)' if score2 > 0.5 else 'Normal (real)'}")


Single image spoof probability: 0.2388
Single image classification: Normal (real)
Multiple images spoof probability: 0.9603
Multiple images classification: Spoof (fake)
